In [5]:
import requests
import os
import uuid
import time
import sys
from dotenv import load_dotenv

# Mude para False quando quiser rodar 
MODO_TESTE = True 

# URL doservidor no Render 
WEBHOOK_URL = "https://weebhook-atriacapital.onrender.com/webhook"

# Intervalo para respeitar 50 requisicoes por minuto (60s / 50 = 1.2s)
INTERVALO_ENTRE_PEDIDOS = 1.2 

sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases
from parametros import Parametros

In [ ]:
# No início do script (antes do loop de disparos)
def acordar_servidor():
    print("Acordando o servidor no Azure...", end=" ")
    try:
        # Chama a raiz do site só pra ele ligar os motores
        requests.get("https://seu-app-nome.azurewebsites.net/", timeout=30)
        print("Servidor Online! 🚀")
    except:
        print("Servidor demorou, mas deve ter acordado.")

# Chame isso antes de começar o loop for
acordar_servidor()

# --- DENTRO DO LOOP ---
for i, conta in enumerate(meus_clientes, 1):
    # ... seu código de pedir relatório ...
    
    # TRUQUE DE MESTRE:
    # A cada 10 contas (aprox 12 segundos), mande um sinal de vida
    if i % 10 == 0:
        try:
            requests.get("https://seu-app-nome.azurewebsites.net/", timeout=1)
        except:
            pass # Não importa o retorno, só o fato de chamar já reseta o timer de hibernação
            
    time.sleep(INTERVALO_ENTRE_PEDIDOS)

In [ ]:
try:
    load_dotenv()
except:
    load_dotenv(encoding='utf-16')

CLIENT_ID = os.getenv("btg_client_id")
CLIENT_SECRET = os.getenv("btg_client_secret")

def get_token():
    url = "https://api.btgpactual.com/iaas-auth/api/v1/authorization/oauth2/accesstoken"
    headers = {
        'x-id-partner-request': str(uuid.uuid4()),
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    }
    data = {'grant_type': 'client_credentials'}
    
    try:
        r = requests.post(url, data=data, headers=headers, auth=(CLIENT_ID, CLIENT_SECRET))
        if r.status_code == 200:
            return r.headers.get('access_token')
        else:
            print(f"ERRO AO GERAR TOKEN: {r.status_code} - {r.text}")
            return None
    except Exception as e:
        print(f"ERRO DE CONEXAO NO TOKEN: {e}")
        return None

def pedir_relatorio(token, conta, dt_ini, dt_fim):
    url = "https://api.btgpactual.com/iaas-profitability/api/v1/performance-report/account"
    
    headers = {
        'x-id-partner-request': str(uuid.uuid4()),
        'access_token': token,
        'Content-Type': 'application/json'
    }
    
    body = {
        "accountNumber": conta,
        "startDate": dt_ini,
        "endDate": dt_fim,
        "webhookService": "performance-report",
        "callbackUrl": WEBHOOK_URL 
    }
    
    print(f"Disparando conta {conta}...", end=" ")
    
    try:
        r = requests.post(url, headers=headers, json=body)
        
        if r.status_code == 202 or r.status_code == 200:
            print(f"OK (Status {r.status_code})")
        elif r.status_code == 429:
            print("RATE LIMIT ATINGIDO. Pausando por 60 segundos...")
            time.sleep(60)
        else:
            print(f"FALHA {r.status_code}: {r.text}")
            
    except Exception as e:
        print(f"ERRO DE REQUEST: {e}")

def carregar_e_filtrar_contas():
    print("Conectando ao banco de dados...")
    conexao = Connect.connect_techdb()
    
    # Importa a tabela inteira como DataFrame
    df_base = Connect.import_table(conexao, "base_btg")
    conexao.close()
    
    print(f"Total de registros na base bruta: {len(df_base)}")

    # ATENCAO: Ajuste 'conta' para o nome exato da coluna no seu banco
    coluna_contas = df_base['Conta'].tolist() 
    
    contas_filtradas = []
    
    # Logica de filtro: Remove se tiver qualquer letra (A-Z)
    for c in coluna_contas:
        c_str = str(c).strip()
        
        # Verifica se existe alguma letra na string
        tem_letra = any(char.isalpha() for char in c_str)
        
        if not tem_letra and len(c_str) > 0:
            contas_filtradas.append(c_str)
            
    return contas_filtradas

if __name__ == "__main__":
    
    # 1. Carrega e filtra a base
    meus_clientes = carregar_e_filtrar_contas()
    
    # --- MODO DE TESTE ---
    if MODO_TESTE:
        print("\n--- MODO DE TESTE ATIVO ---")
        print("Nenhuma requisicao sera enviada ao BTG.")
        print(f"Contas validas encontradas (sem letras): {len(meus_clientes)}")
        print("Exemplo das 10 primeiras contas filtradas:")
        print(meus_clientes[:10])
        print("\nPara executar o envio real, mude MODO_TESTE = False no inicio do script.")
        exit()
    
    # --- MODO DE PRODUCAO ---
    token = get_token()
    
    if token:
        print("\n--- INICIANDO DISPAROS EM PRODUCAO ---")
        print(f"Total de contas: {len(meus_clientes)}")
        print(f"Velocidade estimada: 50 req/min (Pausa de {INTERVALO_ENTRE_PEDIDOS}s)")
        
        # Datas do relatorio
        DT_INI = "2026-01-01"
        DT_FIM = "2026-01-31" 
        
        for i, conta in enumerate(meus_clientes, 1):
            print(f"[{i}/{len(meus_clientes)}]", end=" ")
            pedir_relatorio(token, conta, DT_INI, DT_FIM)
            
            # Pausa obrigatoria para respeitar o limite de 60/min do BTG
            time.sleep(INTERVALO_ENTRE_PEDIDOS)
            
        print("\n--- PROCESSO FINALIZADO ---")
        
    else:
        print("Nao foi possivel obter o Token. Processo abortado.")

Conectando ao banco de dados...
Total de registros na base bruta: 1246

--- MODO DE TESTE ATIVO ---
Nenhuma requisicao sera enviada ao BTG.
Contas validas encontradas (sem letras): 1191
Exemplo das 10 primeiras contas filtradas:
['246022', '264671', '290492', '299305', '321039', '328924', '405159', '449225', '451601', '470497']

Para executar o envio real, mude MODO_TESTE = False no inicio do script.

--- INICIANDO DISPAROS EM PRODUCAO ---
Total de contas: 1191
Velocidade estimada: 50 req/min (Pausa de 1.2s)
[1/1191] Disparando conta 246022... OK (Status 202)
[2/1191] Disparando conta 264671... OK (Status 202)
[3/1191] Disparando conta 290492... OK (Status 202)
[4/1191] Disparando conta 299305... OK (Status 202)
[5/1191] Disparando conta 321039... OK (Status 202)
[6/1191] Disparando conta 328924... OK (Status 202)
[7/1191] Disparando conta 405159... 

KeyboardInterrupt: 

: 